<a href="https://colab.research.google.com/github/bxck75/colab_projects/blob/master/Latest_stablev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cartoon style on image folder



## Setup

In [0]:
# set video details

# video_name='orbital_the_box' 
# vid='https://www.youtube.com/watch?v=IcQXy4YdFcM' # orbital - the box
# video_name='oizo' 
# vid="https://www.youtube.com/watch?v=qmsbP13xu6k" # oizo 
# video_name='techno_dark_'
# vid='https://www.youtube.com/watch?v=lQnzf5vrK5U'
# video_name='techno_dark_2'
# vid='https://www.youtube.com/watch?v=aAMduklK3iU'
# video_name/='right_here' 
# vid="https://www.youtube.com/watch?v=ub747pprmJ8" # righthere
# video_name='this_land' 
# vid="https://www.youtube.com/watch?v=FZN-yJv82Rs"
# video_name="our_love" 
# vid="https://www.youtube.com/watch?v=eyGWBTCZ3mk" # our love 
# video_name='owe_u' 
# vid="https://www.youtube.com/watch?v=OFIWstcaoLc"
# video_name='rhcp'
# vid="https://www.youtube.com/watch?v=GLvohMXgcBo"
# video_name='beatbox1'
# vid="https://www.youtube.com/watch?v=oBjGRZgHgIA"
video_name='beatbox'
vid="https://www.youtube.com/watch?v=oBjGRZgHgIA"

# set Bash vars
# !vidb={vid}
# !video_name={video_name}
# sync google drive
from google.colab import drive
import os
# root = '/content/drive/My Drive/' # google drive root
root = '/content/Cartoonizer/'

if not os.path.exists(root):
   os.mkdir(root,777)

if not os.path.exists(root+video_name+'_frames'):
  os.mkdir(root+video_name+'_frames',777)
  
# if drive is needed uncomment
# drive.mount('/content/drive',force_remount=True)
os.chdir(root)

## Imports!

In [0]:
# imports
import time,os,glob,cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output, display, Image

from moviepy.editor import VideoFileClip
#apt sheit
!apt update
!apt install tree

# clear_output()
repo = "CartoonGan-tensorflow"
!git clone https://github.com/mnicnc404/{repo}.git
os.chdir(os.path.join(repo))
!pip install youtube-dl imageio
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install ffmpeg Imageio
!git clone https://www.github.com/keras-team/keras-contrib.git \
    && cd keras-contrib \
    && python convert_to_tf_keras.py \
    && USE_TF_KERAS=1 python setup.py install
  
# from subprocess import call
import tensorflow as tf
tf.__version__

# clear_output()
#function defines
def resize(image,window_height = 800):
    aspect_ratio = float(image.shape[1])/float(image.shape[0])
    window_width = window_height/aspect_ratio
    image = cv2.resize(image, (int(window_height),int(window_width)))
    return image



In [0]:
!pip freeze > /content/list_modules.lst

## Rip the video and audio

In [0]:
# frame rate
framerate = 0.15
# set max img height
max_size= 500
print('FRAMERATE = '+str(round(1/framerate,2)))

# Set i/o folders
img_in_folder =  root+video_name +"_frames"
img_out_folder = root+"processed_"+video_name +"_frames"

# set original filenames
video_file= root+video_name +'.mp4'
audio_file= root+video_name +'.mp3'

print(video_file)
print(audio_file)
# !rm -r '{audio_file}'
# !rm -r '{video_file}'
# rip video from youtube
print(vid)
os.system('echo youtube-dl '+vid+' --list-formats')
!youtube-dl {vid} --output '{video_file}' --format mp4

clip = VideoFileClip(video_file)
clip.audio.write_audiofile(root+video_name+".mp3")


#capture the video frames
vidcap = cv2.VideoCapture(video_file)
# count total frames
import imageio
vidc = imageio.get_reader(video_file,  'ffmpeg')
num_frames=vidc._meta['nframes']
# make the frames folder
if not os.path.exists(root+video_name+'_frames'):
  os.mkdir(root+video_name+'_frames')
  
# frame grab function
def getFrame(sec):
  vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
  hasFrames,image = vidcap.read()
  if hasFrames:
    cv2.imwrite(root+video_name+"_frames/image"+str(count)+".jpg", image)     # save frame as JPG file
  return hasFrames

sec = 0
FrameRate = framerate
count=1
success = getFrame(sec)
# As long as there are frames 
while success:
  count = count + 1
  frames_left = num_frames -1
  print('frames to go ' + str(frames_left))
  sec = sec + FrameRate
  sec = round(sec, 2)
  success = getFrame(sec)

## Set the style 

In [0]:
# Pick a style
styles = "hosoda" #@param ["shinkai", "hayao", "hosoda", "paprika"]

## Set Dimensions

In [0]:
# generate the movie sequence
searchedfiles = glob.glob(img_in_folder+'/*.jpg')
files = sorted( searchedfiles, key = lambda file: os.path.getctime(file))

files_sorted=[]
combined_sorted=[]
# fill the frame list before the processed image even beeing there
for i in files:
  # split off the filename
  isa = i.split('/')
  # append it with the processed folder path and file definition
  combined_sorted.append("file '"+root+"processed_"+video_name +"_frames/comparison/"+isa[len(isa)-1]+"'\n")
  files_sorted.append("file '"+root+"processed_"+video_name +"_frames/"+styles+"/"+isa[len(isa)-1]+"'\n")
print(combined_sorted)
print(files_sorted)

# for file in files check height
for f in files:
  img = cv2.imread(f)
  h,w,c = img.shape
  # Resize if needed
  if h > max_size:
    img_resized = resize(img,window_height = max_size)
    rh,rw,rc = img_resized.shape
    cv2.imwrite(f,img_resized)

## Run the Cartoonizer

In [0]:
# !ls
# os.chdir('CartoonGan-tensorflow')
# run the Cartoonizerrrrrrrrrrrrrrrrrrrr
!python cartoonize.py \
  --comparison_view horizontal \
  --keep_original_size \
  --batch_size 8 \
  --styles '{styles}' \
  --input_dir '{img_in_folder}' \
  --output_dir '{img_out_folder}'


Transforming:   6% 43/720 [07:32<1:54:56, 10.19s/it, File=image616.jpg]

## Forming the new video 

In [0]:
with open(root+'/paths.txt', 'w') as in_files:
  in_files.writelines(fn for fn in files_sorted)
with open(root+'/compare_paths.txt', 'w') as in_files:
  in_files.writelines(fn for fn in combined_sorted)
  
!ffmpeg -y -r 1/{framerate} -f concat -safe 0 -i  '{root}'"paths.txt" -c:v libx264 -vf "fps=1/"{framerate}",format=yuv420p" '{root}'"processed_"{video_name}".mp4"
!ffmpeg -i '{root}'"processed_"'{video_name}'".mp4" -i '{audio_file}' -shortest '{root}'"final_processed_"'{video_name}'"_vid.mp4"

!ffmpeg -y -r 1/{framerate} -f concat -safe 0 -i  '{root}'"compare_paths.txt" -c:v libx264 -vf "fps=1/"{framerate}",format=yuv420p" '{root}'"processed_compare_"{video_name}".mp4"
!ffmpeg -i '{root}'"processed_compare_"'{video_name}'".mp4" -i '{audio_file}' -shortest '{root}'"final_processed_compare_"'{video_name}'"_vid.mp4"

## View results

In [0]:
folders = [f for f in glob.glob(img_out_folder + "**/*", recursive=True)]
for d in folders:
  !tree '{d}'
  files = [f for f in glob.glob( d + "**/*.jpg", recursive=True )]
  for f in files:
      display(Image(f))


In [0]:

import moviepy
from moviepy.editor import VideoFileClip, clips_array, vfx
print(dir( moviepy.editor))
# def invert_green_blue(image):
#     return image[:,:,[0,2,1]]
# def invert_red_green(image):
#     return image[:,[0,2,1],:

clip1 = VideoFileClip('/content/Cartoonizer/final_processed_compare_'+ video_name + "_vid.mp4").margin(1) # add 10px contour
clip1 = clip1.fx( vfx.colorx, 1.3)
# clip2 = clip1.fx( vfx.mirror_y)
# clip3 = clip1.fx( vfx.mirror_x)
# clip4 = clip1.fx( vfx.mirror_y)
# clip5 = clip1.fx( vfx.colorx, 1.3) # up 40%
# modifiedClip1 = clip1.fl_image( invert_red_green )
# modifiedClip2 = clip1.fl_image( invert_green_blue )
final_clip = clips_array([[clip1]])
final_clip.write_videofile("/content/Cartoonizer/fx_stack"+video_name +".mp4")




In [0]:

from google.colab import drive
import os
video_name='trainrolsin'
drive.mount('/content/drive')
if not os.path.exists("/content/drive/My Drive/"+video_name):
  os.mkdir("/content/drive/My Drive/"+video_name,777)
  os.mkdir("/content/drive/My Drive/"+video_name+"_frames",777)
 
!mv '/content/Cartoonizer/processed_{video_name}_frames/*' '/content/drive/My Drive/'{video_name}'_frames/'

In [0]:
# %from moviepy.editor import *
# clip = (VideoFileClip( root+ video_name " + ".mp4")
# #         .fx( vfx.resize, width=460) # resize (keep aspect ratio)
# #         .fx( vfx.speedx, 2) # double the speed
#         .fx( vfx.colorx, 1.3)) # darken the picture

# clip.write_videofile("brighness_video_name.mp4")

In [0]:
# from moviepy.editor import VideoFileClip, concatenate_videoclips
# clip1 = VideoFileClip( root + video_name " + ".mp4")
# clip2 = VideoFileClip( root + video_name " + ".mp4").subclip(50,60)
# clip3 = VideoFileClip( root + video_name " + ".mp4")
# # final_clip = concatenate_videoclips([clip1,clip2,clip3])
# final_clip = clips_array([[clip1, clip2],
#                           [clip3, clip4]])
# final_clip.write_videofile(video_name+"_concatenation.mp4")